# Mange dataframes and trials
------------------



## Environment

In [2]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [ ]:
import shutil

for idx, row in voxc12_si_fix_df.iterrows():
    wrong_path = "/dataset/SV_sets/voxceleb2/feats/xvector_npy/" + row.feat
    file_name = '-'.join(wrong_path.rsplit('-', 2)[-2:])
    folder_path = "/dataset/SV_sets/voxceleb2/feats/xvector_npy/" + row.spk
    correct_path = folder_path + '/' + file_name
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path)
    try:
        shutil.move(wrong_path, correct_path)
    except FileNotFoundError:
        print(wrong_path)

## Voxceleb 1

### Parsing directory

In [54]:
voxc1_dev = "/dataset/SV_sets/voxceleb1/wav/voxc1_dev/"
voxc1_test = "/dataset/SV_sets/voxceleb1/wav/voxc1_test/"

dev_wav_files = []
for dirpath, dirnames, filenames in os.walk(voxc1_dev):
    if len(filenames) == 0: continue
    dev_wav_files += list(map(lambda x: os.path.join(dirpath, x), filenames))

test_wav_files = []
for dirpath, dirnames, filenames in os.walk(voxc1_test):
    if len(filenames) == 0: continue
    test_wav_files += list(map(lambda x: os.path.join(dirpath, x), filenames))

### Parsed data to dataframes

In [69]:
dev_df = pd.DataFrame(dev_wav_files, columns=['path'])
dev_df['spk']  = dev_df.path.apply(lambda x: x.split('/')[-3])
dev_df['session']  = dev_df.path.apply(lambda x: x.split('/')[-2])
dev_df['file'] = dev_df.path.apply(lambda x: '/'.join(x.split('/')[-3:])[:-4])
dev_df = dev_df.drop(columns="path")
dev_df = dev_df.set_index(dev_df.file.apply(lambda x: '-'.join(x.split('/')[-3:]).rstrip('.path')))
dev_df.index.name = None
dev_df['id'] = dev_df.index

In [120]:
test_df = pd.DataFrame(test_wav_files, columns=['path'])
test_df['spk']  = test_df.path.apply(lambda x: x.split('/')[-3])
test_df['session']  = test_df.path.apply(lambda x: x.split('/')[-2])
test_df['file'] = test_df.path.apply(lambda x: '/'.join(x.split('/')[-3:])[:-4])
test_df = test_df.drop(columns="path")
test_df = test_df.set_index(test_df.file.apply(lambda x: '-'.join(x.split('/')[-3:]).rstrip('.path')))
test_df.index.name = None
test_df['id'] = test_df.index

### Split dev set into dev_train, dev_val, dev_test sets

In [58]:
voxc1_meta = pd.read_csv("/dataset/SV_sets/voxceleb12/vox1_meta.csv")

In [106]:
iden_split = pd.read_csv("/dataset/SV_sets/voxceleb1/iden_split.txt", delimiter=' ', names=['set', 'path'])
iden_split['id'] = iden_split.path.apply(lambda x: x.replace('/', '-')[:-4])
iden_split = iden_split.set_index('id')

In [114]:
set_info = dev_df.id.apply(lambda x: iden_split.loc[x].set)
dev_df['set'] = set_info

### Trials

In [134]:
voxc1_sv_trials = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_test_trial.pkl")

In [135]:
voxc1_sv_trials = voxc1_sv_trials.rename(columns={'enroll_idx':'enr_id', 'test_idx':'test_id', 'enrolment_id':'enr_embed_idx',
                               'test_id':'test_embed_idx'})

### Save dataframes

In [138]:
dev_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_si_dataset.csv")

In [139]:
test_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_sv_dataset.csv")

### Kaldi files

#### utt2spk, spk2utt

TODO: sorting order로 저장해야한다!!

In [ ]:
spks = df.spk.unique().tolist()
spk2utt_dict = dict()
for spk in spks:
    spk2utt_dict[spk] = ' '.join(df[df.spk == spk].index.tolist())

with open("/dataset/SV_sets/voxceleb1/spk2utt", "w") as f:
    for spk, utt in spk2utt_dict.items():
        f.write(spk+" "+utt+"\n")

with open("/dataset/SV_sets/voxceleb1/utt2spk", "w") as f:
    for idx, row in df.iterrows():
        f.write(idx+" "+row.spk+"\n")

### wav.scp

In [ ]:
df = pd.DataFrame(dev_wav_files, columns=['wav'])
df = df.set_index(df.wav.apply(lambda x: '-'.join(x.split('/')[-3:]).rstrip('.wav')))
df.index.name = None

with open("/dataset/SV_sets/voxceleb1/wav.scp", "w") as f:
    for idx, row in df.iterrows():
        f.write(idx+' '+row.wav+'\n')

## voxceleb2

In [122]:
# voxc2 trial 기준 dataframe
voxc2_sv_test_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc2_sv_test_dataframe.pkl")

In [136]:
voxc2_sv_test_df

,spk,feat,label
id10001-1zcIwhmdeo4-00001,id10001,id10001/1zcIwhmdeo4-00001.npy,6114
id10001-1zcIwhmdeo4-00002,id10001,id10001/1zcIwhmdeo4-00002.npy,6114
id10001-1zcIwhmdeo4-00003,id10001,id10001/1zcIwhmdeo4-00003.npy,6114
id10001-7gWzIy6yIIk-00001,id10001,id10001/7gWzIy6yIIk-00001.npy,6114
id10001-7gWzIy6yIIk-00002,id10001,id10001/7gWzIy6yIIk-00002.npy,6114
id10001-7gWzIy6yIIk-00003,id10001,id10001/7gWzIy6yIIk-00003.npy,6114
id10001-7gWzIy6yIIk-00004,id10001,id10001/7gWzIy6yIIk-00004.npy,6114
id10001-7w0IBEWc9Qw-00001,id10001,id10001/7w0IBEWc9Qw-00001.npy,6114
id10001-7w0IBEWc9Qw-00002,id10001,id10001/7w0IBEWc9Qw-00002.npy,6114
id10001-7w0IBEWc9Qw-00003,id10001,id10001/7w0IBEWc9Qw-00003.npy,6114


### trials

In [ ]:
voxc2_all_trial_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc2_trials/voxc2_all_trials.pkl")
voxc2_all_trial_df['enroll_spk']     = voxc2_all_trial_df.enroll_idx.apply(lambda x: x[:7])
voxc2_all_trial_df['test_spk']       = voxc2_all_trial_df.test_idx.apply(lambda x: x[:7])
voxc2_all_trial_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc2_trials/voxc2_all_trials.pkl")

In [ ]:
voxc2_hard_trial_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc2_trials/voxc2_hard_trial.pkl")
voxc2_hard_trial_df['enroll_spk']     = voxc2_all_trial_df.enroll_idx.apply(lambda x: x[:7])
voxc2_hard_trial_df['test_spk']       = voxc2_all_trial_df.test_idx.apply(lambda x: x[:7])
voxc2_hard_trial_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc2_trials/voxc2_hard_trials.pkl")

In [ ]:
voxc2_all_trial_df['is_target'] = voxc2_all_trial_df.label.apply(lambda x: 'target' if x==1 else 'nontarget')
voxc2_all_trial_df.to_csv("/dataset/SV_sets/voxceleb12/kaldi_files/voxc2_all_sv", sep=' ', header=False, index=False, columns=['enroll_idx', 'test_idx', 'is_target'])

In [ ]:
voxc2_hard_trial_df['is_target'] = voxc2_hard_trial_df.label.apply(lambda x: 'target' if x==1 else 'nontarget')
voxc2_hard_trial_df.to_csv("/dataset/SV_sets/voxceleb12/kaldi_files/voxc2_hard_sv", sep=' ', header=False, index=False, columns=['enroll_idx', 'test_idx', 'is_target'])

### utt2spk spk2utt

In [ ]:
spk2utt = voxc2_si_train_df.sort_values("spk").groupby("spk").apply(lambda x: x.uttr_id.values)

In [ ]:
with open("/dataset/SV_sets/gcommand/kaldi_files/spk2utt", "w") as f:
    for k, v in spk2utt.iteritems():
        line = ' '.join([k]+v.tolist())
        f.write(line+'\n')

In [ ]:
utt2spk = all_gcommand_df.sort_values("uttr_id")['spk']

In [ ]:
with open("/dataset/SV_sets/gcommand/kaldi_files/utt2spk", "w") as f:
    for k, v in utt2spk.iteritems():
        line = ' '.join([k, v])
        f.write(line+'\n')

## Voxceleb 1 + 2

### voxc12 dataframes

In [ ]:
voxc12_dev = "/dataset/SV_sets/voxceleb12/feats//"
voxc12_test = "/dataset/SV_sets/voxceleb12/feats/mfcc30_test/"

In [ ]:
def append_dir(file):
    return os.path.join(dirpath, file)

In [ ]:
dev_npy_files = []
for dirpath, dirnames, filenames in os.walk(voxc12_dev):
    if len(filenames) == 0: continue
    dev_npy_files += list(map(append_dir, filenames))

test_npy_files = []
for dirpath, dirnames, filenames in os.walk(voxc12_test):
    if len(filenames) == 0: continue
    test_npy_files += list(map(append_dir, filenames))

In [ ]:
# voxc12_df = pd.DataFrame(dev_npy_files, columns=['wav'])
voxc12_df = pd.DataFrame(test_npy_files, columns=['wav'])

In [ ]:
voxc12_df['spk']  = voxc12_df.wav.apply(lambda x: x.split('/')[-2])
voxc12_df = voxc12_df.set_index(voxc12_df.wav.apply(lambda x: '-'.join(x.split('/')[-2:]).rstrip('. npy')))
voxc12_df.index.name = None
voxc12_df['feat'] = voxc12_df.wav.apply(lambda x: '/'.join(x.split('/')[-2:]))
voxc12_df = voxc12_df.drop(columns="wav")

voxc12_spks = voxc12_df.spk.unique().tolist()

In [ ]:
voxc12_df['label'] = voxc12_df.groupby('spk').ngroup()

In [ ]:
voxc12_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_sv_test_dataframe.pkl")

### voxc1_dev dataframe

In [ ]:
voxc12_dev_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_si_train_dataframe.pkl")
voxc12_dev_spks = voxc12_dev_df.spk.unique().tolist()

In [ ]:
voxc1_dev_spks = [spk for spk in voxc12_dev_spks if spk.startswith('id1')]
voxc1_dev_df = voxc12_dev_df[voxc12_dev_df.spk.isin(voxc1_dev_spks)]

In [ ]:
voxc1_dev_df.loc[:, 'label'] = voxc1_dev_df.groupby('spk').ngroup()

In [ ]:
voxc1_dev_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_si_train_dataframe.pkl")

#### session label


In [ ]:
voxc1_dev_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_si_train_dataframe.pkl")

In [ ]:
valid_session = voxc1_dev_df.session.value_counts()[voxc1_dev_df.session.value_counts() > 7].index.tolist()

In [ ]:
voxc1_dev_df[voxc1_dev_df.session.isin(valid_session)]

### speaker identificaiton data split 

In [ ]:
voxc12_dev_df['session'] = voxc12_dev_df.index

In [ ]:
voxc12_dev_df['session'] = voxc12_dev_df.session.apply(lambda x: x[8:-6])

In [ ]:
voxc12_dev_test_sessions = voxc12_dev_df.groupby('spk').apply(lambda x: x.session.sample(n=1))

In [ ]:
voxc12_dev_df['set'] = 'train'

In [ ]:
voxc12_dev_df.loc[voxc12_dev_df.session.isin(voxc12_dev_test_sessions), 'set'] = 'val'

In [ ]:
len(voxc12_dev_df[voxc12_dev_df.set == 'val']) / len(voxc12_dev_df)

In [ ]:
voxc12_dev_df.to_pickle('/dataset/SV_sets/voxceleb12/dataframes/voxc12_si_train_dataframe.pkl')

### gender split

In [ ]:
voxc1_dev_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_si_train_dataframe.pkl")
voxc1_meta = pd.read_csv("/dataset/SV_sets/voxceleb12/vox1_meta.csv")
voxc1_meta = voxc1_meta.set_index('VoxCeleb1 ID')
voxc1_dev_df['gender'] = voxc1_dev_df.spk.apply(lambda x: voxc1_meta.loc[x].Gender)
voxc1_dev_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_si_train_dataframe.pkl")

In [ ]:
voxc1_test_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_sv_test_dataframe.pkl")
voxc1_meta = pd.read_csv("/dataset/SV_sets/voxceleb12/vox1_meta.csv")
voxc1_meta = voxc1_meta.set_index('VoxCeleb1 ID')
voxc1_test_df['gender'] = voxc1_test_df.spk.apply(lambda x: voxc1_meta.loc[x].Gender)
voxc1_test_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_sv_test_dataframe.pkl")

In [ ]:
voxc1_dev_m = voxc1_dev_df[voxc1_dev_df.gender == 'm']
voxc1_dev_f = voxc1_dev_df[voxc1_dev_df.gender == 'f']

In [ ]:
voxc1_dev_m['label'] = voxc1_dev_m.groupby('spk').ngroup()
voxc1_dev_f['label'] = voxc1_dev_f.groupby('spk').ngroup()

In [ ]:
voxc1_dev_m.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_m_si_train_dataframe.pkl")
voxc1_dev_f.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_f_si_train_dataframe.pkl")

In [ ]:
voxc1_meta.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc1_meta.pkl")

### trial

여러 trial이 있지만 일단은 voxc1의 sv_set을 사용하는 trial을 공용 trial로 지정했다.

In [ ]:
voxc1_test_df = pd.read_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_sv_test_dataframe.pkl") # id의 기준이 되는 df
voxc1_test_spks = voxc1_test_df.spk.unique().tolist()

In [ ]:
voxc1_trial = pd.read_csv("/dataset/SV_sets/voxceleb12/trial_data/veri_test.txt", sep=' ', header=None)
voxc1_trial.columns = ['label', 'enroll', 'test']

In [ ]:
enroll_idx = voxc1_trial.enroll.apply(lambda x: x.replace('/', '-').rstrip('.wav'))
test_idx = voxc1_trial.test.apply(lambda x: x.replace('/', '-').rstrip('.wav'))

voxc1_trial['enroll_idx'] = enroll_idx
voxc1_trial['test_idx'] = test_idx

In [ ]:
enroll_num_idx = voxc1_trial.enroll_idx.apply(lambda x: voxc1_test_df.index.get_loc(x))
test_num_idx = voxc1_trial.test_idx.apply(lambda x: voxc1_test_df.index.get_loc(x))

voxc1_trial['enrolment_id'] = enroll_num_idx
voxc1_trial['test_id'] = test_num_idx

In [ ]:
voxc1_trial = voxc1_trial.drop(columns=['enroll', 'test'])

In [ ]:
voxc1_trial.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_test_trial.pkl")

plda를 위한 trial file

In [ ]:
voxc1_trial['is_target'] = voxc1_trial.label.apply(lambda x: 'target' if x==1 else 'nontarget')

In [ ]:
voxc1_trial.to_csv("/dataset/SV_sets/voxceleb12/trials/voxceleb12_sv", sep=' ', header=False, index=False, columns=['enroll_idx', 'test_idx', 'is_target'])

#### female-female case를 많이 추가한 trial

In [ ]:
voxc1_m_spks = voxc1_meta[voxc1_meta.Gender == 'm'].index.tolist()

In [ ]:
voxc1_f_spks = voxc1_meta[voxc1_meta.Gender == 'f'].index.tolist()

In [ ]:
voxc1_test_m_spks = [spk for spk in voxc1_test_spks if spk in voxc1_m_spks]

In [ ]:
voxc1_test_f_spks = [spk for spk in voxc1_test_spks if spk in voxc1_f_spks]

In [ ]:
# positive pair
import itertools
pos_trials = []
pos_len = dict()
for spk in voxc1_test_spks:
    utters = voxc1_test_df[voxc1_test_df.spk == spk].index
    every_pos_pair = list(filter(diff_session, itertools.combinations(utters, 2)))
    pos_trials += every_pos_pair
    pos_len[spk] = len(every_pos_pair) 

In [ ]:
def diff_session(pair):
    a, b = pair
    if a[8:19] != b[8:19]:
        return True
    else:
        return False

In [ ]:
# negative m pair
import itertools
import random
neg_m_trials = []
for spk in voxc1_test_m_spks:
    utters = voxc1_test_df[ (voxc1_test_df.spk == spk)].index.tolist()
    neg_targets = voxc1_test_df[(voxc1_test_df.gender == 'm') & (voxc1_test_df.spk != spk)].index.tolist()
    total_pairs = list(itertools.product(utters, neg_targets))
    neg_m_trials += list(filter(diff_session, total_pairs))

In [ ]:
# negative f pair
import itertools
import random
neg_f_trials = []
for spk in voxc1_test_f_spks:
    utters = voxc1_test_df[ (voxc1_test_df.spk == spk)].index.tolist()
    neg_targets = voxc1_test_df[(voxc1_test_df.gender == 'f') & (voxc1_test_df.spk != spk)].index.tolist()
    total_pairs = list(itertools.product(utters, neg_targets))
    neg_f_trials += list(filter(diff_session, total_pairs))

In [ ]:
pos_trials = random.choices(pos_trials, k=10000)

In [ ]:
neg_m_trials = random.choices(neg_m_trials, k=int(len(pos_trials)/2))

In [ ]:
neg_f_trials = random.choices(neg_f_trials, k=int(len(pos_trials)/2))

In [ ]:
neg_trials = neg_m_trials + neg_f_trials

In [ ]:
voxc1_trials = pos_trials + neg_trials
voxc1_trial_df = pd.DataFrame(voxc1_trials, columns=["enroll_idx", "test_idx"])

In [ ]:
voxc1_trial_df['enroll_uttr_id'] = voxc1_trial_df.enroll_idx.apply(lambda x: voxc1_test_df.index.get_loc(x))
voxc1_trial_df['test_uttr_id'] = voxc1_trial_df.test_idx.apply(lambda x: voxc1_test_df.index.get_loc(x))
voxc1_trial_df['enroll_spk'] = voxc1_trial_df.enroll_idx.apply(lambda x: x[:7])
voxc1_trial_df['test_spk'] = voxc1_trial_df.test_idx.apply(lambda x: x[:7])
voxc1_trial_df['label'] = [1]*len(pos_trials) + [0]*len(neg_trials)
voxc1_trial_df.to_pickle("/dataset/SV_sets/voxceleb12/dataframes/voxc12_bal_sv_test_dataframe.pkl")

In [ ]:
voxc1_trial_df['is_target'] = voxc1_trial_df.label.apply(lambda x: 'target' if x==1 else 'nontarget')
voxc1_trial_df.to_csv("/dataset/SV_sets/voxceleb12/trials/voxceleb12_bal_sv", sep=' ', header=False, index=False, columns=['enroll_idx', 'test_idx', 'is_target'])

## gcommand 

In [ ]:
gcommand_train_df = pd.read_pickle("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/equal_num_30spk_si.pkl")
gcommand_test_df = pd.read_pickle("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/equal_num_30spk_sv.pkl")

In [ ]:
val_uttrs_idx = gcommand_train_df.groupby('spk', group_keys=False).apply(lambda x: x.sample(n=2)).index

In [ ]:
gcommand_train_df.set = 'train'

In [ ]:
gcommand_train_df.label

In [ ]:
gcommand_train_df.loc[val_uttrs_idx, 'set'] = 'val'

In [ ]:
gcommand_train_df.set.value_counts()

In [ ]:
gcommand_train_df.to_pickle("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/equal_num_30spk_si.pkl")

In [ ]:
all_gcommands_df = pd.read_pickle("/dataset/SV_sets/gcommand/dataframes/all_gcommands.pkl")

### utt2spk spk2utt

In [ ]:
spk2utt = gcommand_train_df.sort_values("spk").groupby("spk").apply(lambda x: x.uttr_id.values)

In [ ]:
with open("/dataset/SV_sets/gcommand/kaldi_files/train_spk2utt", "w") as f:
    for k, v in spk2utt.iteritems():
        line = ' '.join([k]+v.tolist())
        f.write(line+'\n')

In [ ]:
utt2spk = gcommand_train_df.sort_values("uttr_id")['spk']

In [ ]:
with open("/dataset/SV_sets/gcommand/kaldi_files/train_utt2spk", "w") as f:
    for k, v in utt2spk.iteritems():
        line = ' '.join([k, v])
        f.write(line+'\n')

### trials (self-made)

In [ ]:
gcommand_test_spks = gcommand_test_df.spk.unique().tolist()

In [ ]:
# positive pair
import itertools
pos_trials = []
pos_len = []
for spk in gcommand_test_spks:
    utters = gcommand_test_df[gcommand_test_df.spk == spk].index
    every_pos_pair = itertools.combinations(utters, 2)
    pos_trials += every_pos_pair

In [ ]:
# positive pair
import itertools
import random
neg_trials = []
for spk in gcommand_test_spks:
    utters = gcommand_test_df[gcommand_test_df.spk == spk].index.tolist()
    neg_targets = gcommand_test_df[gcommand_test_df.spk != spk].index.tolist()
    total_pairs = list(itertools.product(utters, neg_targets))
    neg_trials += random.choices(total_pairs, k=5995)

In [ ]:
gcommand_trials = pos_trials + neg_trials
gcommand_trial_df = pd.DataFrame(gcommand_trials, columns=["enroll_idx", "test_idx"])

In [ ]:
gcommand_trial_df['enrolment_id'] = gcommand_trial_df.enroll_idx.apply(lambda x: gcommand_test_df.index.get_loc(x))

gcommand_trial_df['test_id'] = gcommand_trial_df.test_idx.apply(lambda x: gcommand_test_df.index.get_loc(x))

gcommand_trial_df['enroll_spk'] = gcommand_trial_df.enroll_idx.apply(lambda x: x[:8])

gcommand_trial_df['test_spk'] = gcommand_trial_df.test_idx.apply(lambda x: x[:8])

gcommand_trial_df['label'] = [1]*len(pos_trials) + [0]*len(neg_trials)

gcommand_trial_df.to_pickle("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/equal_num_30spk_test_trial.pkl")

In [ ]:
gcommand_trial_df['is_target'] = gcommand_trial_df.label.apply(lambda x: 'target' if x==1 else 'nontarget')

gcommand_trial_df.to_csv("/dataset/SV_sets/gcommand/kaldi_files/gcommand_30spks_sv", sep=' ', header=False, index=False, columns=['enroll_idx', 'test_idx', 'is_target'])

In [ ]:
gcommand_train_df = gcommand_train_df.sort_index()

In [ ]:
gcommand_train_df['label'] = gcommand_train_df.groupby('spk').ngroup()

In [ ]:
gcommand_train_df.to_pickle("/dataset/SV_sets/gcommand/dataframes/equal_num_30spk/equal_num_30spk_si.pkl")